In [1]:
import pandas as pd
import json

import os

## For `question_answer_pairs` dataset

In [2]:
def read_question_answer_pairs_info(qa_dataset_fp, article_fp):
    '''
    Read information regarding relationship between article, questions and answers
    Returning list of questions and answers row
    Example output: [
     'Alessandro_Volta\tWas Volta an Italian physicist?\tyes\teasy\teasy\tS09_set4_a10\n',
     'Alessandro_Volta\tIs Volta buried in the city of Pittsburgh?\tno\teasy\teasy\tS09_set4_a10\n'
    ]
    '''
    s = []
    for file in os.listdir(qa_dataset_fp):
        if file.endswith('.txt'):
            print("Read file "+file)
            with open(qa_dataset_fp+'/{}'.format(file), 'r', encoding='utf-8', errors='ignore') as f:
                temp_s = f.readlines()
                s.extend(temp_s)
    return s

def build_dataset(qa_dataset_fp, article_fp):
    s = read_question_answer_pairs_info(qa_dataset_fp, article_fp)
    
    document_names = os.listdir(article_fp)
    in_json_list = []
    for article_file in document_names:
        in_json = {}
        with open(
            '{}/{}'.format(article_fp, article_file), 'r', encoding='utf-8', errors='ignore') as f:
            in_json['context'] = f.read()
        in_json['id'] = article_file.replace('.txt.clean','')
        in_json['qas'] = []
        in_json_list.append(in_json)
    print("There are {} documents.".format(len(in_json_list)))
    
    out_json = {}
    doc_id_list = [x['id'] for x in in_json_list]
    
    for i,each_s in enumerate(s[1:]):
        # Extract information of each row, exclude first row as the first row is the column name
        cont = each_s.split('\t')
        qas_json_list = []
        qa_json = {}
        
        cont = each_s.split('\t')
        ref_file = cont[-1].replace('\n','')
        qa_json['question'] = cont[1]
        qa_json['answers'] = cont[2]
        qa_json['id'] = i

        # Store the questions and answers pair into corresponding document(context)
        for idx,doc_id in enumerate(doc_id_list):
            if doc_id == ref_file:
                in_json_list[idx]['qas'].append(qa_json)
    dummy = {}
    dummy['paragraphs'] = in_json_list
    out_json['data'] = [dummy]
    
    return out_json

In [3]:
out_json = build_dataset('data/questionanswer-dataset', 'data/questionanswer-dataset/text_data')

with open('input/test.json','w') as f:
    json.dump(out_json, f)

Read file S08_question_answer_pairs.txt
Read file S09_question_answer_pairs.txt
Read file S10_question_answer_pairs.txt
There are 165 documents.


END